# <div style="text-align: center; color: #1a5276;">Graph Mode</div>

## <font color='blue'> Table of Contents </font>

1. [Introduction](#1)
2. [Setup](#2)  
3. [Simple examples](#3)
4. [Print behavior](#4)
5. [Training](#5)
6. [References](#references)


<a name="1"></a>
## <font color='blue'> 1. Introduction </font>

Graph Mode is a powerful feature where your computations are first defined as a computation graph before being executed. This mode is used primarily in TensorFlow 1.x and still available in TensorFlow 2.x through @tf.function.

Key benefits:

- Performance Optimization
    - Since the graph is static, TensorFlow can apply various optimizations like constant folding, kernel fusion, etc., improving execution speed.

- Deployment Efficiency 
    - Graphs can be serialized, exported, and run on different platforms (e.g., mobile devices, TPU, etc.).

- Parallel Execution
    - The static graph enables parallel computation across devices efficiently.

- Error Checking
    - Graphs are analyzed before execution, allowing TensorFlow to catch potential issues early.


For debugging or quick prototyping, Eager Execution is generally more intuitive, but Graph Mode excels in production scenarios requiring efficiency.

Graph Mode: When you use @tf.function, TensorFlow traces the function once (using a sample input or the first value you provide), creating a static computation graph. This graph includes all operations needed for forward and backward passes (like matrix multiplications, activations, gradients, etc.). It doesn’t dynamically change once traced — meaning the structure is fixed, but the values computed during the forward and backward passes will depend on the input data and current model parameters.

Eager Execution: When not using @tf.function, TensorFlow operates in eager execution mode, which means that operations are evaluated immediately as you run the code, and the graph is built dynamically as the operations are executed. This allows for more flexibility but is slower than graph execution.

<a name="2"></a>
## <font color='blue'> 2. Setup </font>

In [27]:
import os  
import time  
import numpy as np  
import tensorflow as tf  

In [28]:
seed = 42

np.random.seed(seed)                    
tf.random.set_seed(seed)                

<a name="3"></a>
## <font color='blue'> 3. Simple Examples </font>

### Adding tensors

In [5]:
@tf.function  # Converts to Graph Mode
def add_tensors(x, y):
    return x + y

# Example usage
a = tf.constant(3)
b = tf.constant(5)
print(add_tensors(a, b))  # Output: 8

tf.Tensor(8, shape=(), dtype=int32)


Let's see the generted code:

- tf.autograph.to_code(): This function takes a Python function as input and returns the generated TensorFlow graph code (in Python syntax) that TensorFlow uses internally to optimize execution.


- add_tensors.python_function: Since add_tensors is a @tf.function-decorated function, .python_function accesses the original Python implementation before TensorFlow compiles it.

In [7]:
# See what the generated code looks like
print(tf.autograph.to_code(add_tensors.python_function))

def tf__add_tensors(x, y):
    with ag__.FunctionScope('add_tensors', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = ag__.ld(x) + ag__.ld(y)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



### Visualizing the computation graph

We can use TensorBoard to visualize the computation graph.

In [9]:
# Define your function with @tf.function to enable graph mode
@tf.function
def add_tensors(x, y):
    return x + y

# Create a directory for TensorBoard logs
log_dir = "logs/graph"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Create a SummaryWriter to log the graph
writer = tf.summary.create_file_writer(log_dir)

# Log the graph to the writer
with writer.as_default():
    # Log the graph using tf.summary.trace_on() and tf.summary.trace_export()
    tf.summary.trace_on(graph=True)
    a = tf.constant(3)
    b = tf.constant(5)
    add_tensors(a, b)
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)

# Start TensorBoard (in the terminal or an IDE with TensorBoard support)
# Run the following in your terminal:
# tensorboard --logdir=logs/graph


If we open http://localhost:6006 in our browser we can view the graph under the Graph tab in TensorBoard.

<img src="images/ComputationGraph.png"/>

### Control flows

Control flow statements which are very intuitive to write in eager mode can look very complex in graph mode. 

In [10]:
# simple function that returns the square if the input is greater than zero
@tf.function
def f(x):
    if x>12:
        x = x * x
    return x

print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def if_body():
            nonlocal x
            x = ag__.ld(x) * ag__.ld(x)

        def else_body():
            nonlocal x
            pass
        ag__.if_stmt(ag__.ld(x) > 12, if_body, else_body, get_state, set_state, ('x',), 1)
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



### Common errors

The following code will raise an exception:

In [29]:
import tensorflow as tf

@tf.function
def simple_model(x):
    try:
        weights = tf.Variable([[0.1, 0.2], [0.3, 0.4]])
        biases = tf.Variable([0.5, 0.6])
        output = tf.matmul(x, weights) + biases
        return tf.nn.relu(output)
    except ValueError as e:
        tf.print("Error:", e)
        tf.print("Tip: Define `weights` and `biases` outside the `@tf.function` block to avoid this issue.")

# Example input data
x = tf.constant([[1.0, 2.0]])

# Forward pass
simple_model(x)


Error: ValueError('tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.')
Tip: Define `weights` and `biases` outside the `@tf.function` block to avoid this issue.


The error occurs because TensorFlow's @tf.function decorator requires that tf.Variable objects are created outside the decorated function or only once during the first call. When you define weights and biases inside the @tf.function-decorated function, TensorFlow tries to recreate these variables every time the function is called, which is not allowed.

To fix this, you need to create the tf.Variable objects once, outside of the function, and then use them inside the @tf.function function.

In [12]:
# Define the model with tf.function
@tf.function
def simple_model(x, weights, biases):
    output = tf.matmul(x, weights) + biases
    return tf.nn.relu(output)

# Create variables outside the tf.function
weights = tf.Variable([[0.1, 0.2], [0.3, 0.4]])
biases = tf.Variable([0.5, 0.6])

# Example input data
x = tf.constant([[1.0, 2.0]])

# Forward pass
print(simple_model(x, weights, biases))


tf.Tensor([[1.2 1.6]], shape=(1, 2), dtype=float32)


In the previous code:
    
- The structure of the graph is defined once when you call simple_model(x) for the first time.

- Values like weights, and biases will change as you pass new data during training.

- The graph stays the same, but the values for activations and gradients will change depending on the data and current model parameters.

<a name="4"></a>
## <font color='blue'> 4. Print behavior </font>

Consider the following code:

In [20]:
## print
def f(x):
    print("Traced with", x)

for i in range(5):
    f(2)


Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 2


The previous code shows the normal behaviour of the print function. Now, consider the following code:

In [22]:
@tf.function
def f(x):
    print("Traced with", x)

for i in range(5):
    f(2)
    

Traced with 2


Even though the loop runs 5 times, the print statement only executes once. To solve this, we need to use tf.print() instead of print().

In [23]:
@tf.function
def f(x):
    print("Traced with", x)
    # added tf.print
    tf.print("Executed with", x)

for i in range(5):
    f(2)


Traced with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2


**Key takeaway:** tf.print is graph aware and will run as expected in loops.

<a name="5"></a>
## <font color='blue'> 5. Training </font>

We will train a simple model with and without graph mode and compare the execution time.


In [26]:
# Define a simple model for binary classification
class SimpleModel(tf.keras.Model):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(64, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Loss and optimization functions
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Generate random data for training (1000 samples, 20 features)
X_train = np.random.randn(1000, 20).astype(np.float32)
y_train = np.random.randint(0, 2, size=(1000, 1)).astype(np.float32)

# Create an instance of the model
model = SimpleModel()

# Training step without @tf.function
def train_step_no_graph(x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_fn(y, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training step with @tf.function (graph mode)
@tf.function
def train_step_with_graph(x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_fn(y, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Helper function to train for one epoch
def train_epoch(train_data, train_labels, train_step_fn):
    total_loss = 0
    for x_batch, y_batch in zip(train_data, train_labels):
        # Ensure x_batch has the correct shape (batch_size, num_features)
        if x_batch.ndim == 1:
            x_batch = np.expand_dims(x_batch, axis=0)
        # Ensure y_batch has the correct shape (batch_size, 1)
        if y_batch.ndim == 1:
            y_batch = np.expand_dims(y_batch, axis=0)
        loss = train_step_fn(x_batch, y_batch)
        total_loss += loss
    return total_loss / len(train_data)

# Fix the input shape by ensuring X_train has a batch dimension
if X_train.ndim == 1:
    X_train = X_train.reshape(-1, 20)  # Ensures input is (batch_size, 20)

# Train with no graph mode (normal Python function)
start_time = time.time()
train_epoch(X_train, y_train, train_step_no_graph)
end_time = time.time()
no_graph_duration = end_time - start_time
print(f"Training with no graph mode took {no_graph_duration:.4f} seconds.")

# Train with graph mode (tf.function)
start_time = time.time()
train_epoch(X_train, y_train, train_step_with_graph)
end_time = time.time()
graph_duration = end_time - start_time
print(f"Training with graph mode took {graph_duration:.4f} seconds.")

# Compare the results
print(f"Speedup from graph mode: {no_graph_duration / graph_duration:.2f}x")

Training with no graph mode took 13.1743 seconds.
Training with graph mode took 1.0698 seconds.
Speedup from graph mode: 12.31x


The results show a significant performance boost using graph mode. The training time decreased from 13.17s to 1.07s, achieving a 12.31x speedup. This highlights TensorFlow's efficiency in optimizing code execution with graph mode.

<a name="6"></a>
## <font color='blue'> 6. Summary </font>

**Summary of Key Benefits:**

- Performance Boost: Faster execution through optimization, better memory management, and reduced Python overhead.

- Hardware Optimization: Efficient execution on GPUs/TPUs with hardware-specific optimizations.

- Portability: Easier deployment and execution across different platforms and hardware.

- Reduced Python Overhead: Compiles operations into a static graph, eliminating the need for repeated function calls in Python.

- Efficient Backpropagation: Optimized gradient computation, leading to faster training.

By using @tf.function, you're enabling TensorFlow to optimize execution for performance, which is especially important for complex models and large datasets.


<a name="references"></a>
## <font color='blue'> References </font>

[TF Advanced Techniques](https://www.coursera.org/specializations/tensorflow-advanced-techniques)